In [ ]:
# reupload all data to have as one dataset again
completedf1 = pd.concat([pd.read_csv('0208plays.csv'),pd.read_csv('0914plays.csv')])

#(remove 2002, because it has served it's purpose of creating 2003 priors)
completedf = completedf1[completedf1['year']>2002].reset_index(drop=True)

In [ ]:
print completedf.shape
completedf.head(2)

lcols = ['distToGoal',"inDown3","inDown2","inDown1",'inQuarter4',"inQuarter3","inQuarter2","inQuarter1",'secsElapsedInHalf',"margin","yds_to_go","addscore","tm_TO_left","opp_TO_left","inFGRange", "inRedZone", "tm_winprob","inLast3minHalf", "tm_prev_yr_pass","tm_in_season_pass","tm_in_game_pass"]

In [ ]:
def masking(df,itrain,itest):    
    mask=np.ones(df.shape[0], dtype='int')
    mask[itrain]=1
    mask[itest]=0
    mask = (mask==1)
    return mask

In [ ]:
def runsvm(Xmatrix_train, Yresp_train, Cs):
    from sklearn.svm import LinearSVC
    clfsvm_lin=LinearSVC(loss="hinge")
    
    from sklearn.grid_search import GridSearchCV
    gs=GridSearchCV(clfsvm_lin, param_grid={'C':Cs}, cv=5)
    gs.fit(Xmatrix_train, Yresp_train)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    
    best = gs.best_estimator_
    print best.fit(Xmatrix_train, Yresp_train)
    print best.score(Xmatrix_test, Yresp_test)
    
    return best
    

def runsvc(Xmatrix_train, Yresp_train, Cs):
    from sklearn.svm import SVC
    clfsvm_orig = SVC()
    gs_orig = GridSearchCV(clfsvm_orig, param_grid={'C':Cs}, cv=5)
    gs_orig.fit(Xmatrix_train, Yresp_train)
    print "BEST", gs_orig.best_params_, gs_orig.best_score_, gs_orig.grid_scores_
    
    best_orig = gs_orig.best_estimator_
    print best_orig.fit(Xmatrix_train, Yresp_train)
    print best_orig.score(Xmatrix_test, Yresp_test)
    
    return best_orig

In [29]:
def cv_optimize(clf, parameters, X, y, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best

In [30]:
from sklearn.metrics import confusion_matrix
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.6f" % (training_accuracy)
    print "Accuracy on test data:     %0.6f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [ ]:
def randforest(df,lcols,mask):    
    from sklearn.ensemble import RandomForestClassifier
    clfForest = RandomForestClassifier()
    parameters = {"n_estimators": range(1,40)}
    # could add njobs to the mix to run them in parallel
    clfForest, Xtrain, ytrain, Xtest, ytest = do_classify(clfForest, parameters, df, lcols, u'RESP', 1, mask=mask, score_func='f1')
    return clfForest

def gradboost(df,lcols,mask):
    from sklearn.ensemble import GradientBoostingClassifier
    clfGB = GradientBoostingClassifier()
    parameters = {"n_estimators": range(1, 20)}
    clfGB, Xtrain, ytrain, Xtest, ytest = do_classify(clfGB, parameters, df, lcols, u'RESP', 1, mask=mask, score_func='f1')
    return clfGB

def adaboost(clfForest,df,lcols,mask):
    from sklearn.ensemble import AdaBoostClassifier
    clfAda = AdaBoostClassifier(base_estimator=clfForest)
    parameters = {"n_estimators": range(1, 20)}
    clfAda, Xtrain, ytrain, Xtest, ytest = do_classify(clfAda, parameters, df, lcols, u'RESP', 1, mask=mask, score_func='f1')
    return clfAda

In [35]:
def plot_features(clfForest,lcols):
    importance_list = clfForest.feature_importances_
    importance_list, name_list = zip(*sorted(zip(importance_list, lcols)))
    plt.barh(range(len(name_list)),importance_list,align='center')
    plt.yticks(range(len(name_list)),name_list)
    plt.xlabel('Relative Importance in the Random Forest')
    plt.ylabel('Features')
    plt.title('Relative Importance of Each Feature')
    plt.show()
    

def plot_decision_surface(clf, X_train, Y_train):
    plot_step=0.1
    
    if X_train.shape[1] != 2:
        raise ValueError("X_train should have exactly 2 columnns!")
    
    x_min, x_max = X_train[:, 0].min() - plot_step, X_train[:, 0].max() + plot_step
    y_min, y_max = X_train[:, 1].min() - plot_step, X_train[:, 1].max() + plot_step
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step), np.arange(y_min, y_max, plot_step))

    clf.fit(X_train,Y_train)
    if hasattr(clf, 'predict_proba'):
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
    else:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])    
    
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Reds)
    plt.scatter(X_train[:,0],X_train[:,1],c=Y_train,cmap=plt.cm.Paired)
    plt.show()
    
    print type(X_train[:, 0].min()), type(plot_step)
    

def plot_rf(clfForest,clflog,lcols,df):
    xt = [lcols[e] for e in clfForest.feature_importances_.argsort()[::-1][0:2]]
    X_imp = df[xt].values
    Y = df.RESP.values

    classifiers = [clfForest, clflog]

    titleClassifer = ['Random Forest Classifier', "Logistic Regression Classifier"]
    for c in range(len(classifiers)):
        plt.title(titleClassifer[c])
        plt.xlabel(xt[0])
        plt.ylabel(xt[1])
        plot_decision_surface(classifiers[c], X_imp, Y)

In [ ]:
def model(completedf,year):
    # use the dataframe for the input year
    df = completedf[completedf['year']==year]
    
    #Split into train and test, create mask
    itrain, itest = train_test_split(xrange(df.shape[0]), train_size=0.7)
    mask = masking(df,itrain,itest)
    
    # initialize variables for svm/svc
    Cs=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0] # try fewer if doesnt finish up
    Xmatrix=df[lcols].values
    Yresp=df['RESP'].values
    Xmatrix_train=Xmatrix[mask]
    Xmatrix_test=Xmatrix[~mask]
    Yresp_train=Yresp[mask]
    Yresp_test=Yresp[~mask]
    
    # run svm  
    best = runsvm(Xmatrix_train, Yresp_train,Cs)
    
    # run svc    
    best_orig = runsvc(Xmatrix_train, Yresp_train,Cs)
    
    # run logistic models
    from sklearn.linear_model import LogisticRegression
    clflog,_,_,_,_ = do_classify(LogisticRegression(penalty="l1"), {"C": [0.001, 0.01, 0.1, 1, 10, 100]}, df, lcols, u'RESP', 1, mask=mask)
    clflog2,_,_,_,_ = do_classify(LogisticRegression(penalty="l2"), {"C": [0.001, 0.01, 0.1, 1, 10, 100]}, df, lcols, u'RESP', 1, mask=mask)
    
    #Random forest classifier and make random forest plots
    clfForest = randforest(df,lcols,mask)
    plot_features(clfForest,lcols)
    plot_rf(clfForest,clflog,lcols,df)
    
    # ADA Boost Classifier
    clfAda = adaboost(clfForest,df,lcols,mask)
    
    # Gradient Boost CLassifier
    clfGB = gradboost(df,lcols,mask)

    return best, best_orig, clflog, clflog2, clfForest, clfAda, clfGB

In [ ]:
clfsvm, clfsvc, clflog, clflog2, clfForest, clfAda, clfGB = model(completedf, 2014)